1. Scrap the news articles from yahoofinance for interested Ticker and date
2. Save it in dataframe
3. Summarize  each articles using NLP
4. Calculate the sentiment of the summarized articles

In [1]:
#!pip install yahoo_fin pyarrow requests_html lxml_html_clean

In [2]:
import numpy as np
import pandas as pd
import torch
import requests
import re
from bs4 import BeautifulSoup
from transformers import pipeline
import concurrent.futures
import unicodedata
import json
from transformers import AutoTokenizer, AutoModelForSequenceClassification

/Users/sudipkhadka/Desktop/Sentiment-Analysis/.sen/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from yahoo_fin import news

In [4]:
#sentiment_pipeline = pipeline("sentiment-analysis", model='distilbert/distilbert-base-uncased-finetuned-sst-2-english') default

In [5]:
#sentiment_pipeline = pipeline("sentiment-analysis", model='yiyanghkust/finbert-tone')



In [6]:
sentiment_pipeline = pipeline("sentiment-analysis", model='nickmuchi/fb-bart-large-finetuned-trade-the-event-finance-summarizer')

Some weights of BartForSequenceClassification were not initialized from the model checkpoint at nickmuchi/fb-bart-large-finetuned-trade-the-event-finance-summarizer and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use mps:0


In [1]:
summarizer = pipeline('summarization')

NameError: name 'pipeline' is not defined

In [8]:
def get_article_text(url):
    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            paragprahs = soup.find_all('p')
            article_text = ' '.join([p.get_text() for p in paragprahs])
            article_text = basic_cleanup(article_text)
            return article_text
        else:
            print(f"Failed to get article text from {url} got a response code of {response.status_code}")
            return "N/A"
    except Exception as e:
        print(f"Failed to get article text from {url}: {e}")
        return "N/A"

In [9]:
stock_news = news.get_yf_rss("NVDA")
df = pd.DataFrame(stock_news)
df.head()

,summary,summary_detail,id,guidislink,links,link,published,published_parsed,title,title_detail
0,It would be an understatement to say that Pala...,"{'type': 'text/html', 'language': None, 'base'...",a3149ade-9521-33ee-8d8d-3e18e1baaf00,False,"[{'rel': 'alternate', 'type': 'text/html', 'hr...",https://www.fool.com/investing/2024/12/06/wher...,"Fri, 06 Dec 2024 22:00:00 +0000","(2024, 12, 6, 22, 0, 0, 4, 341, 0)",Where Will Palantir Technologies Stock Be in 1...,"{'type': 'text/plain', 'language': None, 'base..."
1,"The Nasdaq is closing in on the 20,000 level b...","{'type': 'text/html', 'language': None, 'base'...",15a2cb08-33be-325d-bfd4-c341e781aa60,False,"[{'rel': 'alternate', 'type': 'text/html', 'hr...",https://finance.yahoo.com/m/15a2cb08-33be-325d...,"Fri, 06 Dec 2024 21:45:38 +0000","(2024, 12, 6, 21, 45, 38, 4, 341, 0)","Dow Jones Futures: Nasdaq Nears 20,000 But Wat...","{'type': 'text/plain', 'language': None, 'base..."
2,"AppLovin is just one of three ""aptly"" named hi...","{'type': 'text/html', 'language': None, 'base'...",422d6d71-3f6d-36ca-9315-fb68a08b430a,False,"[{'rel': 'alternate', 'type': 'text/html', 'hr...",https://finance.yahoo.com/m/422d6d71-3f6d-36ca...,"Fri, 06 Dec 2024 21:07:32 +0000","(2024, 12, 6, 21, 7, 32, 4, 341, 0)",The 'Apps' Have It — So Does Palantir And 26 O...,"{'type': 'text/plain', 'language': None, 'base..."
3,SoundHound stock rose Thursday and Friday afte...,"{'type': 'text/html', 'language': None, 'base'...",09c4654c-c30c-3419-a728-7459a994ea3b,False,"[{'rel': 'alternate', 'type': 'text/html', 'hr...",https://finance.yahoo.com/m/09c4654c-c30c-3419...,"Fri, 06 Dec 2024 21:06:35 +0000","(2024, 12, 6, 21, 6, 35, 4, 341, 0)",Nvidia-backed SoundHound Notches 61% Weekly Ga...,"{'type': 'text/plain', 'language': None, 'base..."
4,"Dubbed the Magnificent Seven stocks, Apple, Mi...","{'type': 'text/html', 'language': None, 'base'...",4205eaa9-f620-3a0b-a81a-0e82c7c9fd0b,False,"[{'rel': 'alternate', 'type': 'text/html', 'hr...",https://finance.yahoo.com/m/4205eaa9-f620-3a0b...,"Fri, 06 Dec 2024 20:45:42 +0000","(2024, 12, 6, 20, 45, 42, 4, 341, 0)",Magnificent Seven Stocks: Nvidia Stock Slides;...,"{'type': 'text/plain', 'language': None, 'base..."


In [10]:
df['summary'][0]

'It would be an understatement to say that Palantir Technologies (NASDAQ: PLTR) stock has been in fine form on the market in 2024, as shares of the software platform specialist have shot up a stunning 290% so far this year as of this writing.  The past month alone has been a terrific one for Palantir investors as the stock has zoomed 62% since releasing its third-quarter results on Nov. 4.  Artificial intelligence (AI) has played a defining role in this red-hot rally as enterprises and governments have been flocking to Palantir to help them integrate generative AI into their operations, helping the company accelerate its growth and build a robust revenue pipeline.'

In [11]:
df['title'][0]

'Where Will Palantir Technologies Stock Be in 1 Year?'

In [12]:
df['summary'][1]

'The Nasdaq is closing in on the 20,000 level but is starting to get extended. Four tech titans are flashing buy signals.'

## Sentiment Scoring

FinBERT is a pre-trained NLP model to analyze sentiment of financial text. It is built by further training the BERT language model in the finance domain, using a large financial corpus and thereby fine-tuning it for financial sentiment classification.

In [13]:
tokenizer = AutoTokenizer.from_pretrained('ProsusAI/finbert')
model = AutoModelForSequenceClassification.from_pretrained('ProsusAI/finbert')

In [14]:
def sentiment_score(news):
    tokens = tokenizer.encode(news, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [15]:
sentiment_score(df['summary'][5])

3

In [16]:
df['summary'][3]

"SoundHound stock rose Thursday and Friday after the company inked a deal with Torchy's Tacos to use its voice AI technology."

In [17]:
df['market Sentiments'] = df['summary'].apply(lambda x: sentiment_score(x))


In [18]:
df[['market Sentiments','summary']]

,market Sentiments,summary
0,1,It would be an understatement to say that Pala...
1,3,"The Nasdaq is closing in on the 20,000 level b..."
2,3,"AppLovin is just one of three ""aptly"" named hi..."
3,1,SoundHound stock rose Thursday and Friday afte...
4,1,"Dubbed the Magnificent Seven stocks, Apple, Mi..."
5,3,We recently published a list of 14 AI News and...
6,1,The company has partnered with Nvidia to deplo...
7,2,EU Investigates Nvidia for Potential Antitrust...
8,3,Fast-food companies are joining other industri...
9,3,(Updates with Nvidia's response in the sixth p...


In [19]:
df['market Sentiments'] = df['market Sentiments'].apply(
    lambda x: 'Positive' if x == 3 else 'Neutral' if x==2 else 'Negative'
)

In [20]:
df[['market Sentiments','summary']]

,market Sentiments,summary
0,Negative,It would be an understatement to say that Pala...
1,Positive,"The Nasdaq is closing in on the 20,000 level b..."
2,Positive,"AppLovin is just one of three ""aptly"" named hi..."
3,Negative,SoundHound stock rose Thursday and Friday afte...
4,Negative,"Dubbed the Magnificent Seven stocks, Apple, Mi..."
5,Positive,We recently published a list of 14 AI News and...
6,Negative,The company has partnered with Nvidia to deplo...
7,Neutral,EU Investigates Nvidia for Potential Antitrust...
8,Positive,Fast-food companies are joining other industri...
9,Positive,(Updates with Nvidia's response in the sixth p...


In [21]:
market_sentiment = df['market Sentiments'].mode()[0]
print(f'Market Sentiment is: {market_sentiment}')

Market Sentiment is: Positive


## Now use all the summarized articles and give the final summary to back up your sentement.

In [22]:
all_summary = " ".join(df['summary'].astype(str).tolist())

In [23]:
summary_result = summarizer(all_summary, max_length=200, min_length=50, do_sample=False)

Token indices sequence length is longer than the specified maximum sequence length for this model (1055 > 1024). Running this sequence through the model will result in indexing errors


In [24]:
summary_result

[{'summary_text': ' Palantir Technologies (NASDAQ: PLTR) stock has been in fine form on the market in 2024, as shares of the software platform specialist have shot up a stunning 290% so far this year . The past month alone has been a terrific one for investors as the stock has zoomed 62% since releasing its third-quarter results on Nov. 4 . The United States government recently announced fresh curbs targeting the chip industry .'}]

In [25]:
from transformers import pipeline

rephraser = pipeline("text2text-generation", model="t5-small")  # Replace with a larger model if needed
summarized_result = summarizer(all_summary, max_length=200, min_length=50, do_sample=False)



Device set to use mps:0


In [26]:
summarized_result

[{'summary_text': ' Palantir Technologies (NASDAQ: PLTR) stock has been in fine form on the market in 2024, as shares of the software platform specialist have shot up a stunning 290% so far this year . The past month alone has been a terrific one for investors as the stock has zoomed 62% since releasing its third-quarter results on Nov. 4 . The United States government recently announced fresh curbs targeting the chip industry .'}]